This is a tutorial notebook on object-oriented software design, prepared for the Foundations of Software Engineering (FSE v2020.1, https://github.com/adasegroup/FSE2020_seminars) at Skoltech (http://skoltech.ru).

Copyright 2020 by Alexey Artemov and ADASE Lab. 

# FSE-08: Object-Oriented Software Design

The purpose of this practical is to give an introduction into object-oriented software design. You will learn to:
 - Formulate real-world problems in object-oriented terms
 - Use Python classes to implement the designed models

### Legend

In these frightening times, it is of utmost importance to be able to model the spread of various viruses. Our government has hired your company to create a flexible computational model for the spread of the most dangerous viruses, the Virus Spread Modelling System VSMS-20.

In this task, your goal is to design an implement models of these viruses:

 - Seasonal flu virus
 - SARS-CoV-2
 - Cholera 
 
The goal is to be able to perform simulations to assess efficiency of public policymaking aiming to save lives.

### **What do we need to model?**

In this design, we need to model the following domain entities as objects:
 1. Viruses and infections, responsible for infecting persons.
 2. Persons, responsible for being infected, transmitting infections.
 3. The Department of Health, responsible for healthcare policymaking.
 4. Hospitals, responsible for treating persons and performing tests.
 
We need to model the following processes:
 1. Person getting infected by the virus.
 2. Person going through a number of states (e.g. healthy - asymptomatic infected - symptomatic infected - recovered).
 3. Person getting tested for a speficic type of virus.
 4. Person getting hospitalized.
 5. Hospital figuring out the treatment for a person based on tests.
 6. Department of Health checking for the number of infections, as reported by positive tests.
 7. Department of Health establishing a policy (e.g., social distancing, full/partial lockdown, or mandatory wearing masks).

We assume the following environmental constraints:
 - Persons cannot be born during the course of the simulation (but may become dead)
 - Persons can only have 1 virus (but many antibodies)
 - Persons may or may not obey the policies of the government
 - The Department of Health does not penalize persons for not obeying the policies

**The purpose of this part of the practical** is to figure out the preliminary design of such a system. 

---

### 1. Model viruses/infections

#### Task 1:

Create class hierarchy for viruses/infections:
 1. Define the abstract class `Infectable` with a method `cause_symptoms` and two fields:
   - `strength`, i.e. the ability of the virus to cause more severe symptoms;
   - `contagiousness`, i.e. the ability of the virus to transmit from person to person.
 2. Derive three other classes `SeasonalFluVirus`, `SARSCoV2`, and `Cholera`, leaving the implementation empty for now.

<img src="images/Infectable Class.svg">

#### Solution to task 1:

In [ ]:
# YOUR CODE HERE

### 2. Model a person

#### Task 2:

Create a class hierarchy for persons:
 1. Create an abstract class `Person` with the attributes shown in the diagram. Note that:
   - When a person is healthy, they would execute common actions (e.g. go to work and from work), implemented via `day_actions`, `night_actions`, and `interact` with other people if they are `is_contacting`. This may result in a person `get_infected` by a infection.
   - But when a person is sick, they use `fight_virus` and `progress_disease` to describe this situation. Additionally we provide `is_life_threatening_condition` and `is_life_incompatible_condition` to test if a person is in a critical situation.
 2. Create derived classes `DefaultPerson` and `CommunityPerson`, overriding respective methods.

<img src="images/Person Class.svg">

#### Solution to Task 2:

In [ ]:
# YOUR CODE HERE

### 3. Model of a healthcare system and the global context

#### Task 3:

We restrict ourselves to only one country (no international flights, etc.). 

Create the model of the hospital, department of health, and global context:
 1. `Hospital` hosts and treats patients.
 2. `DepartmentOfHealth` monitors the epidemiological situation and can issue orders.
 3. `GlobalContext` contains references to the world grid and a list of persons

<img src="images/Hospital Class.svg">

#### Solution to task 3:

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

### 5. Model the world

The basic algorithm works on a daily basis.
 1. The Department of Health counts positive/negative tests from yesterday, decides if they need to introduce the new policies/restrictions.
 2. The hospitals treat patients, making new tests.
 3. People do their daily activities and interact, possibly infecting each other.

In [ ]:
def simulate_day(context):
    persons, health_dept, hospitals = context.persons, context.health_dept, context.health_dept.hospitals

    health_dept.make_policy()
    
    for hospital in hospitals:
        hospital.treat_patients()
    
    for person in persons:
        person.day_actions()
    
    for person in persons:
        for other in persons:
            if person is not other and person.is_close_to(other):
                person.interact(other)
                
    for person in persons:
        person.night_actions()

We initialise our world by writing a number of functions:

In [ ]:
from random import randint


def create_persons(min_j, max_j, min_i, max_i, n_persons):
    min_age, max_age = 1, 90
    min_weight, max_weight = 30, 120
    persons = [
        Person(
            home_position=(randint(min_j, max_j), randint(min_i, max_i)),
            age=randint(min_age, max_age),
            weight=randint(min_weight, max_weight),
        )
        for i in range(n_persons)
    ]
    return persons


def create_department_of_health(hospitals):
    return DepartmentOfHealth(hospitals)


def create_hospitals(n_hospitals):
    hospitals = [
        Hospital(capacity=100)
        for i in range(n_hospitals)
    ]
    return hospitals


def initialize():
    # our little country
    min_i, max_i = 0, 100
    min_j, max_j = 0, 100
    
    # our citizen
    n_persons = 1000
    persons = create_persons(min_j, max_j, min_i, max_i, n_persons)
        
    # our healthcare system
    n_hospitals = 4
    hospitals = create_hospitals(n_hospitals)
    
    health_dept = create_department_of_health(hospitals)
    
    # global context
    context = GlobalContext(
        (min_j, max_j, min_i, max_i),
        persons,
        health_dept
    )

    return context

In [ ]:
import tqdm

In [ ]:
context = initialize()

for day in tqdm.tqdm(range(100)):
    simulate_day(context)